In [1]:
import pandas as pd
import requests
import json
import base64
import tensorflow as tf

In [2]:
data = pd.read_csv('data/brain_stroke.csv')
inputs = data.iloc[102, :10].to_dict()
inputs

{'gender': 'Male',
 'age': 80.0,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'Yes',
 'work_type': 'Private',
 'Residence_type': 'Rural',
 'avg_glucose_level': 259.63,
 'bmi': 31.7,
 'smoking_status': 'smokes'}

In [3]:
data.iloc[102, :11]

gender                  Male
age                     80.0
hypertension               0
heart_disease              0
ever_married             Yes
work_type            Private
Residence_type         Rural
avg_glucose_level     259.63
bmi                     31.7
smoking_status        smokes
stroke                     1
Name: 102, dtype: object

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://project-mlops-stroke-prediction-production.up.railway.app/v1/models/stroke-classification-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No Stroke" if prediction_value < 0.5 else "Stroke"
else:
    result = "Error: No predictions found in response."

print(result)

Stroke
